## Installing DKube SDK

In [ ]:
! pip3 install git+https://github.com/oneconvergence/dkube.git@2.2 --upgrade --no-cache-dir
import os
os._exit(00)

In [ ]:
from dkube.sdk import *

In [ ]:
user = os.getenv('USERNAME')
authToken = os.getenv('DKUBE_USER_ACCESS_TOKEN')
inp_path = "/opt/dkube/input"
out_path = "/opt/dkube/output"
code_name = "mnist-fs"
training_fs = "mnist-fs"
dataset_name = "mnist"
model_name = "mnist"
framework = "tensorflow_1.14"
image = "ocdr/d3-datascience-tf-cpu:fs-v1.14"
preprocess_script = f"python featureset.py --fs {training_fs}"
training_script = f"python model.py --fs {training_fs}"
transformer_code = "tf/classification/mnist-fs/digits/transformer/transformer.py"

In [ ]:
api = DkubeApi(token=authToken)

In [ ]:
# api.list_models(user = user)

In [ ]:
preprocessing_name= generate(code_name)
preprocessing = DkubePreprocessing(user, name=preprocessing_name, description='triggered from dkube sdk')
preprocessing.update_container(image_url=image)
preprocessing.update_startupscript(preprocess_script)
preprocessing.add_code(code_name)
preprocessing.add_input_dataset(dataset_name, mountpath=inp_path)
preprocessing.add_output_featureset(training_fs, mountpath=out_path)

In [ ]:
api.create_preprocessing_run(preprocessing)

In [ ]:
training_name= generate(code_name)
training = DkubeTraining(user, name=training_name, description='triggered from dkube sdk')
training.update_container(framework=framework, image_url=image)
training.update_startupscript(training_script)
training.add_code(code_name)
training.add_input_featureset(training_fs, mountpath=inp_path)
training.add_output_model(model_name, mountpath=out_path)

In [ ]:
api.create_training_run(training)

In [ ]:
for each_datum in api.list_models(user = user)[0]["datums"]:
    if each_datum["datum"]["name"] == model_name:
        head_version = each_datum["datum"]["generated"]["head_version"]
        break

In [ ]:
head_version

In [ ]:
serving_name=generate(code_name)
serving = DkubeServing(user, name=serving_name, description='serving deployed from dkube sdk')
serving.update_transformer_code(project=code_name, code=transformer_code)
serving.update_transformer_image(image_url='ocdr/d3-datascience-tf-cpu:v1.14')
serving.update_serving_model(model_name, version = head_version)
serving.update_serving_image(image_url='ocdr/tensorflowserver:1.14')
serving.set_production_deploy() #For production deployment, when MODEL_CATALOG_ENABLED=false
serving.update_autoscaling_config(min_replicas=1, max_concurrent_requests=1)

In [ ]:
api.create_test_inference(serving)